Import Libraries

In [1]:
import psycopg2
import google.generativeai as genai
import logging
from dotenv import load_dotenv
import os

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

load_dotenv()

True

Open Database Connection

In [2]:
conn = None
cursor = None

def open_db_connection():
    """Open a connection to the PostgreSQL database."""
    global conn, cursor
    try:
        conn = psycopg2.connect(
            dbname="api_development_db",
            user="postgres",
            password=os.getenv("POSTGRES_PASSWORD", "semah"),  # Replace with your password in .env if different
            host="localhost",
            port="5432"
        )
        cursor = conn.cursor()
        logger.info("Connected to PostgreSQL database")
    except Exception as e:
        logger.error(f"Error connecting to PostgreSQL: {str(e)}")
        raise

open_db_connection()

2025-04-08 00:08:57,269 - INFO - Connected to PostgreSQL database


Fetch CV Text from Database

In [3]:
def fetch_cv_text():
    """Fetch the text for a specific CV from the PostgreSQL database based on filename from .env."""
    try:
        filename = os.getenv("LLM_ANALYSIS_FILENAME", "LLM_ANALYSIS_FILENAME")
        if filename == "LLM_ANALYSIS_FILENAME":
            logger.error("No valid filename defined in LLM_ANALYSIS_FILENAME in .env")
            return None, None
        
        cursor.execute("SELECT filename, text FROM cvs WHERE filename = %s", (filename,))
        result = cursor.fetchone()
        conn.commit()
        
        if not result:
            logger.warning(f"No CV found with filename: {filename}")
            return filename, None
        
        cv_filename, cv_text = result
        logger.info(f"Fetched CV text for filename: {cv_filename}")
        return cv_filename, cv_text
    except Exception as e:
        logger.error(f"Error fetching CV text: {str(e)}")
        conn.rollback()
        return None, None

cv_filename, cv_text = fetch_cv_text()

2025-04-08 00:09:31,306 - INFO - Fetched CV text for filename: juriste.png


Analyze with LLM

In [4]:
def analyze_with_llm(text_data):
    """Analyze text data using Google's Gemini API to extract skills, experiences, and qualifications."""
    try:
        api_key = os.getenv("GEMINI_API_KEY", "GEMINI_API_KEY")
        if api_key == "GEMINI_API_KEY":
            logger.error("No valid GEMINI_API_KEY defined in .env")
            return {}

        genai.configure(api_key=api_key)
        
        all_text = " ".join(text_data)
        prompt = os.getenv("LLM_ANALYSIS_PROMPT", "LLM_ANALYSIS_PROMPT").format(text=all_text)
        
        model = genai.GenerativeModel("gemini-1.5-flash")
        response = model.generate_content(prompt)
        
        extracted_text = response.text.strip()
        skills, experiences, qualifications = [], [], []
        lines = extracted_text.split("\n")
        current_section = None
        
        for line in lines:
            line = line.strip()
            if line.startswith("Skills:"):
                current_section = "skills"
                skills = line.replace("Skills:", "").strip().split(", ")
            elif line.startswith("Experiences:"):
                current_section = "experiences"
                experiences = line.replace("Experiences:", "").strip().split(", ")
            elif line.startswith("Qualifications:"):
                current_section = "qualifications"
                qualifications = line.replace("Qualifications:", "").strip().split(", ")
            elif line and current_section:
                if current_section == "skills":
                    skills.extend(line.split(", "))
                elif current_section == "experiences":
                    experiences.extend(line.split(", "))
                elif current_section == "qualifications":
                    qualifications.extend(line.split(", "))

        skills = [skill.strip() for skill in skills if skill.strip()]
        experiences = [exp.strip() for exp in experiences if exp.strip()]
        qualifications = [qual.strip() for qual in qualifications if qual.strip()]

        logger.info("Gemini LLM analysis completed successfully")
        return {
            "skills": skills,
            "experiences": experiences,
            "qualifications": qualifications
        }
    except Exception as e:
        logger.error(f"Error during Gemini LLM analysis: {str(e)}")
        return {"skills": [], "experiences": [], "qualifications": []}

if cv_text:
    extracted_data = analyze_with_llm([cv_text])
    logger.info(f"Extracted data for {cv_filename}: {extracted_data}")
else:
    logger.error("No CV text available to analyze")
    extracted_data = {"skills": [], "experiences": [], "qualifications": []}

2025-04-08 00:10:24,978 - INFO - Gemini LLM analysis completed successfully
2025-04-08 00:10:24,981 - INFO - Extracted data for juriste.png: {'skills': ['Contract Negotiation', 'Legal Research'], 'experiences': ['Several years in law firms'], 'qualifications': ['Law Degree', 'Jurisprudence']}


Close Database Connection

In [5]:
def close_db_connection():
    """Close the PostgreSQL database connection."""
    global conn, cursor
    try:
        if conn and conn.closed == 0:
            if conn.status == psycopg2.extensions.STATUS_IN_TRANSACTION:
                conn.rollback()
            cursor.close()
            conn.close()
            logger.info("Database connection closed successfully")
    except Exception as e:
        logger.error(f"Error closing database connection: {str(e)}")

close_db_connection()

2025-04-08 00:11:17,637 - INFO - Database connection closed successfully
